In [1]:
import pandas as pd
from cuotasimss import CuotasImss
from imss import CalculoIMSSObrero, CalculoIMSSPatron
from isr import ISR
from datetime import datetime
from sbc import SBC
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
df = pd.read_excel('./data/plantilla_laboral2.xlsx', sheet_name='Hoja1')
df.head()

,Id,Nombre,Sueldo mensual,Vales,Fecha Ingreso
0,1,Martha Rosa Alvarez Martínez,"33,256.68","1,500.00",2019-06-03
1,2,Tania Monserrat Bañuelos Murillo,"25,931.50","1,297.00",2019-06-17
2,6,Karla Berenice Cano Garcia,"22,960.82","1,148.00",2019-07-23
3,7,Jonathan Covarrubias Sustaita,"21,719.58","1,086.00",2019-03-04
4,8,Miguel Dionicio Luna,"27,315.62","1,366.00",2018-01-09


In [3]:
df['Fecha Ingreso'] = pd.to_datetime(df['Fecha Ingreso'], format='%Y-%m-%d')

In [4]:
date_2022 = pd.to_datetime('2022/12/31')

In [6]:
df['SDI_2022'] = df['Sueldo mensual'].apply(lambda x: x / 30)
df['SBC_2022'] = df.apply(lambda x: SBC.funcion_sbc(sueldo = x['Sueldo mensual'], vales = x['Vales'], a = x['Fecha Ingreso'], b = date_2022), axis=1)
df['ISR_2022'] = (df['Sueldo mensual'] * 1).apply(lambda x: ISR.function_isr(x))
df['IMSS_O_2022'] = df['SBC_2022'].apply(lambda x: CalculoIMSSObrero.function_imss_obrero(x))
df['IMSS_P_2022'] = df.apply(lambda x: CalculoIMSSPatron.function_imss_patronal(x = x['SBC_2022'], date=date_2022 ), axis=1)
df['ISN_2022'] = df['Sueldo mensual'] * 0.03
_isr_ = df['Sueldo mensual'].apply(lambda x: ISR.function_isr(x))
df['Neto'] = df['Sueldo mensual'] - _isr_ - df['IMSS_O_2022']
df['Aguinaldo_2022'] = df.apply(lambda x: SBC.funcion_aguinaldo(sueldo = x['Sueldo mensual'], a = x['Fecha Ingreso'], b = date_2022), axis=1)
df['Prima Vacacional_2022'] = df.apply(lambda x: SBC.funcion_pv(sueldo = x['Sueldo mensual'], date_ingreso = x['Fecha Ingreso'], date_base = date_2022), axis=1)

In [7]:
df.head()

,Id,Nombre,Sueldo mensual,Vales,Fecha Ingreso,SDI_2022,SBC_2022,ISR_2022,IMSS_O_2022,IMSS_P_2022,ISN_2022,Neto,Aguinaldo_2022,Prima Vacacional_2022
0,1,Martha Rosa Alvarez Martínez,"33,256.68","1,500.00",2019-06-03,"1,108.56","1,172.56","5,797.85",954.07,"6,308.16",997.70,"26,504.76","16,628.34","2,771.39"
1,2,Tania Monserrat Bañuelos Murillo,"25,931.50","1,297.00",2019-06-17,864.38,910.00,"4,097.80",732.57,"5,007.64",777.94,"21,101.13","12,965.75","2,160.96"
2,6,Karla Berenice Cano Garcia,"22,960.82","1,148.00",2019-07-23,765.36,802.05,"3,463.27",641.51,"4,472.94",688.82,"18,856.04","11,480.41","1,913.40"
3,7,Jonathan Covarrubias Sustaita,"21,719.58","1,086.00",2019-03-04,723.99,758.70,"3,198.14",604.93,"4,258.19",651.59,"17,916.51","10,859.79","1,809.97"
4,8,Miguel Dionicio Luna,"27,315.62","1,366.00",2018-01-09,910.52,961.87,"4,400.52",776.33,"5,264.55",819.47,"22,138.77","13,657.81","2,731.56"
